In [1]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import DynamicUpdater
from mindsponge import SimulationCell
from mindsponge.control import VelocityVerlet, Langevin, BerendsenBarostat
from mindsponge.function import VelocityGenerator
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.optimizer import SteepestDescent

In [3]:
system = Molecule(template='water.spce.yaml')
system.set_pbc_box([0.4, 0.4, 0.4])

Molecule<>

如果定义了周期性的盒子，那么还可以使用repeat_box功能来扩展/复制系统，比如可以这样将系统复制1000份

In [4]:
system.repeat_box([10, 10, 10])

Molecule<>

得到的分子系统的示意图如下所示：
![](../../docs/tutorial_b05.png)

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = SteepestDescent(system.trainable_params(), 1e-6)

In [7]:
sim = SimulationCell(system, potential, cutoff=1.0)
md = Sponge(sim, optimizer=opt)

In [8]:
run_info = RunInfo(10)
md.run(100, callbacks=[run_info])

Step: 0, E_pot: -706514.3, 
Step: 10, E_pot: -780552.5, 
Step: 20, E_pot: -782711.0, 
Step: 30, E_pot: -782719.3, 
Step: 40, E_pot: -782720.56, 
Step: 50, E_pot: -782722.25, 
Step: 60, E_pot: -782720.2, 
Step: 70, E_pot: -782721.56, 
Step: 80, E_pot: -782719.44, 
Step: 90, E_pot: -782720.7, 


In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.coordinate.shape, system.atom_mass)

In [10]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, temp),
                     barostat=BerendsenBarostat(system, 1),
                     velocity=velocity,
                     time_step=5e-4)
md = Sponge(sim, optimizer=opt)

In [11]:
md.change_optimizer(opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b05.h5md', save_freq=10)

In [13]:
md.run(1000, callbacks=[run_info, cb_h5md])

Step: 0, E_pot: -782718.3, E_kin: 10916.313, E_tot: -771802.0, Temperature: 291.85968, Pressure: 3368.909, Volume: 64.0
Step: 10, E_pot: -779897.06, E_kin: 7632.4365, E_tot: -772264.6, Temperature: 204.0616, Pressure: 1535.7832, Volume: 64.03076
Step: 20, E_pot: -779699.7, E_kin: 8917.413, E_tot: -770782.25, Temperature: 238.41687, Pressure: 2574.6794, Volume: 64.06491
Step: 30, E_pot: -782010.2, E_kin: 10394.465, E_tot: -771615.75, Temperature: 277.90747, Pressure: 4127.326, Volume: 64.10935
Step: 40, E_pot: -780588.3, E_kin: 8327.25, E_tot: -772261.06, Temperature: 222.63821, Pressure: 1256.0485, Volume: 64.15217
Step: 50, E_pot: -781512.7, E_kin: 9897.135, E_tot: -771615.56, Temperature: 264.6108, Pressure: 1443.5708, Volume: 64.17936
Step: 60, E_pot: -783051.3, E_kin: 10603.81, E_tot: -772447.5, Temperature: 283.50452, Pressure: 1090.4902, Volume: 64.20383
Step: 70, E_pot: -782691.9, E_kin: 10302.405, E_tot: -772389.5, Temperature: 275.44617, Pressure: 170.93893, Volume: 64.209404
